**TRAIN Model**

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import time
import numpy as np
import matplotlib.pyplot as plt

# Parameters
model = DualEfficientNetModel(num_classes=2) 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Variables to track accuracy and loss
accuracy_list = []
loss_list = []
num_epochs = 5

# Start time for training
train_start_time = time.time()

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for mel_inputs, mfcc_inputs, labels in train_loader:  # Unpacking dual inputs from dataloader
        mel_inputs = mel_inputs.to(device)
        mfcc_inputs = mfcc_inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass with both inputs
        outputs = model(mel_inputs, mfcc_inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Accuracy calculations
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


    # Calculating epoch loss and accuracy
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    accuracy_list.append(epoch_accuracy)
    loss_list.append(epoch_loss)

    # Printing epoch summary
    print(f"\033[92mEpoch {epoch + 1}/{num_epochs}\n Loss: {epoch_loss:.4f}\n Accuracy: {epoch_accuracy:.2f}%\033[0m")

# End time for training
train_end_time = time.time()
print(f"\033[1;91mTime taken for training: {train_end_time - train_start_time:.2f} seconds\033[0m")

# Plotting Accuracy and Loss per Epoch
plt.figure(figsize=(8, 6))
plt.plot(range(1, num_epochs + 1), accuracy_list, marker='o', label='Accuracy')
plt.plot(range(1, num_epochs + 1), loss_list, marker='x', label='Loss')
plt.title('Training Progress')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.legend()
plt.grid(True)

# Accuracy and loss values on the graph
for i, (acc, loss) in enumerate(zip(accuracy_list, loss_list)):
    plt.annotate(f'{acc:.2f}', (i + 1, acc), textcoords="offset points", xytext=(0, 5), ha='center')
    plt.annotate(f'{loss:.2f}', (i + 1, loss), textcoords="offset points", xytext=(0, -10), ha='center')

plt.show()